In [1]:
from google.colab import files
file_path = files.upload()

Saving dataset_NLP_cs.csv to dataset_NLP_cs.csv


In [2]:
import pandas as pd
import re
import json
df = pd.read_csv('dataset_NLP_cs.csv')
df.head()

,text,code
0,"in the function gcd(a,b=integers)","int gcd(int a, int b) {"
1,"if b=1 return a, else call function gcd(b, a%b)","return !b ? a : gcd(b, a % b);"
2,NaN,}
3,NaN,int main() {
4,"n , nn, ans = integers with ans =0","int n, nn, ans = 0;"


In [3]:
def debug_dataframe(df):
    """
    Debug the DataFrame to understand its structure and content
    """
    print("🔍 DEBUGGING DATAFRAME STRUCTURE")
    print("=" * 50)

    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"DataFrame info:")
    print(df.info())

    print(f"\n📊 COLUMN ANALYSIS:")
    for col in df.columns:
        print(f"\nColumn: '{col}'")
        print(f"  Data type: {df[col].dtype}")
        print(f"  Non-null count: {df[col].count()}")
        print(f"  Null count: {df[col].isna().sum()}")
        print(f"  Empty string count: {(df[col].astype(str).str.strip() == '').sum()}")

        # Sample values
        if len(df) > 0:
            print(f"  Sample values:")
            for i in range(min(3, len(df))):
                value = df.iloc[i][col]
                print(f"    {i+1}. {str(value)[:100]}...")

    print(f"\n🧹 CHECKING FOR SPECIFIC ISSUES:")

    # Check if all values are the same (causing duplicates removal)
    for col in df.columns:
        unique_count = df[col].nunique()
        print(f"  Column '{col}': {unique_count} unique values")
        if unique_count == 1 and len(df) > 0:
            print(f"    ⚠️  All values are identical: '{df[col].iloc[0]}'")

    # Check for whitespace-only values
    for col in df.columns:
        if df[col].dtype == 'object':
            whitespace_only = df[col].astype(str).str.strip().eq('').sum()
            print(f"  Column '{col}': {whitespace_only} whitespace-only values")

# Run debugging
debug_dataframe(df)

🔍 DEBUGGING DATAFRAME STRUCTURE
DataFrame shape: (293854, 2)
Columns: ['text', 'code']
DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293854 entries, 0 to 293853
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    216225 non-null  object
 1   code    293854 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB
None

📊 COLUMN ANALYSIS:

Column: 'text'
  Data type: object
  Non-null count: 216225
  Null count: 77629
  Empty string count: 3
  Sample values:
    1. in the function gcd(a,b=integers)...
    2. if b=1 return a, else call function gcd(b, a%b)...
    3. nan...

Column: 'code'
  Data type: object
  Non-null count: 293854
  Null count: 0
  Empty string count: 0
  Sample values:
    1. int gcd(int a, int b) {...
    2. return !b ? a : gcd(b, a % b);...
    3. }...

🧹 CHECKING FOR SPECIFIC ISSUES:
  Column 'text': 117794 unique values
  Column 'code': 84840 unique values
  Column 'text': 3 whit

In [4]:
def robust_preprocess_dataframe(df):
    """
    More robust preprocessing that handles various data issues
    """
    print("🔄 STARTING ROBUST PREPROCESSING")
    print(f"Original shape: {df.shape}")

    # Make a copy to avoid modifying original
    df_clean = df.copy()

    # 1. Check and rename columns if needed
    expected_columns = ['text', 'code']
    actual_columns = list(df_clean.columns)

    print(f"Actual columns: {actual_columns}")

    # If columns don't match expected names, try to map them
    if 'text' not in df_clean.columns or 'code' not in df_clean.columns:
        print("⚠️  Columns don't match expected names, attempting to map...")

        # Common alternative column names
        column_mapping = {
            'pseudo_code': 'text',
            'pseudocode': 'text',
            'instruction': 'text',
            'description': 'text',
            'python_code': 'code',
            'target': 'code',
            'output': 'code'
        }

        for old_col, new_col in column_mapping.items():
            if old_col in df_clean.columns:
                df_clean[new_col] = df_clean[old_col]
                print(f"  Mapped '{old_col}' -> '{new_col}'")

    # 2. Remove completely empty rows
    initial_count = len(df_clean)
    df_clean = df_clean.dropna(how='all')  # Only drop rows where ALL columns are NaN
    print(f"After dropping completely empty rows: {len(df_clean)}/{initial_count}")

    # 3. Fill partial NaN values with empty strings instead of dropping
    df_clean = df_clean.fillna('')

    # 4. Check if we have the required columns now
    if 'text' not in df_clean.columns or 'code' not in df_clean.columns:
        print("❌ Still missing required columns after mapping.")
        print(f"Available columns: {list(df_clean.columns)}")

        # If we have exactly 2 columns, assume they're text and code in order
        if len(df_clean.columns) == 2:
            col1, col2 = df_clean.columns
            df_clean = df_clean.rename(columns={col1: 'text', col2: 'code'})
            print(f"Assumed column mapping: '{col1}' -> 'text', '{col2}' -> 'code'")
        else:
            return None

    # 5. Remove rows where both text and code are empty
    mask = (df_clean['text'].astype(str).str.strip() != '') | (df_clean['code'].astype(str).str.strip() != '')
    df_clean = df_clean[mask]
    print(f"After removing rows where both text and code are empty: {len(df_clean)}")

    # 6. Remove exact duplicates based on both text and code
    initial_count = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['text', 'code'])
    print(f"After removing exact duplicates: {len(df_clean)}/{initial_count}")

    # 7. Normalize formatting
    print("Normalizing formatting...")

    # Convert to string and clean
    df_clean['text'] = df_clean['text'].astype(str).str.strip()
    df_clean['code'] = df_clean['code'].astype(str).str.strip()

    # Remove rows with very short content (likely errors)
    df_clean = df_clean[df_clean['text'].str.len() >= 5]
    df_clean = df_clean[df_clean['code'].str.len() >= 5]
    print(f"After removing very short entries: {len(df_clean)}")

    # Normalize Python code formatting
    df_clean['code'] = df_clean['code'].apply(lambda x: re.sub(r'\t', '    ', x))  # Tabs to spaces
    df_clean['code'] = df_clean['code'].apply(lambda x: x.replace('\r\n', '\n').replace('\r', '\n'))  # Line endings

    # 8. Create processed data
    processed_data = []

    for idx, row in df_clean.iterrows():
        formatted_text = f"""<|start|>
Pseudo-Code:
{row['text']}

Python Code:
{row['code']}
<|end|>"""

        processed_data.append({"text": formatted_text})

    # Save as JSONL
    if processed_data:
        with open('pseudo2python_dataset.jsonl', 'w', encoding='utf-8') as f:
            for item in processed_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')

        print(f"✅ Preprocessing completed!")
        print(f"📁 Saved {len(processed_data)} examples to 'pseudo2python_dataset.jsonl'")

        # Display samples
        print(f"\n📋 Sample formatted examples:")
        for i in range(min(3, len(processed_data))):
            print(f"\n--- Example {i+1} ---")
            sample_text = processed_data[i]['text']
            # Show first 200 characters to avoid overwhelming output
            print(sample_text[:200] + "..." if len(sample_text) > 200 else sample_text)

    return processed_data

# Run the robust preprocessing
print("Starting robust preprocessing...")
processed_data = robust_preprocess_dataframe(df)

if processed_data:
    print(f"\n🎉 Successfully processed {len(processed_data)} examples!")
else:
    print("\n❌ Preprocessing failed. Please check the column structure of your DataFrame.")

    # Show what we can work with
    print(f"\n💡 SUGGESTIONS:")
    print("1. Check if your DataFrame has columns named 'text' and 'code'")
    print("2. If columns have different names, let me know what they are")
    print("3. Make sure there is actual data in the DataFrame")
    print(f"\nCurrent DataFrame columns: {list(df.columns)}")
    print(f"Current DataFrame shape: {df.shape}")

Starting robust preprocessing...
🔄 STARTING ROBUST PREPROCESSING
Original shape: (293854, 2)
Actual columns: ['text', 'code']
After dropping completely empty rows: 293854/293854
After removing rows where both text and code are empty: 293854
After removing exact duplicates: 123427/293854
Normalizing formatting...
After removing very short entries: 122574
✅ Preprocessing completed!
📁 Saved 122574 examples to 'pseudo2python_dataset.jsonl'

📋 Sample formatted examples:

--- Example 1 ---
<|start|>
Pseudo-Code:
in the function gcd(a,b=integers)

Python Code:
int gcd(int a, int b) {
<|end|>

--- Example 2 ---
<|start|>
Pseudo-Code:
if b=1 return a, else call function gcd(b, a%b)

Python Code:
return !b ? a : gcd(b, a % b);
<|end|>

--- Example 3 ---
<|start|>
Pseudo-Code:
n , nn, ans = integers with ans =0

Python Code:
int n, nn, ans = 0;
<|end|>

🎉 Successfully processed 122574 examples!


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def setup_tokenizer(model_name="gpt2"):
    print("🔧 SETTING UP TOKENIZER...")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Add special tokens
    special_tokens = {
        'additional_special_tokens': [
            '<|start|>',
            '<|end|>',
            'Pseudo-Code:',
            'Python Code:'
        ]
    }

    # Ensure we have a pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Add special tokens
    tokenizer.add_special_tokens(special_tokens)

    print(f"✅ Tokenizer loaded: {model_name}")
    print(f"Vocabulary size: {tokenizer.vocab_size}")
    print(f"Special tokens: {tokenizer.all_special_tokens}")

    return tokenizer

# Initialize tokenizer
tokenizer = setup_tokenizer()

# Test the tokenizer on a sample
sample_text = processed_data[0]['text']
print(f"\n🧪 TOKENIZER TEST:")
print(f"Sample text length: {len(sample_text)} characters")
print(f"Tokenized length: {len(tokenizer.encode(sample_text))} tokens")

# Show tokenization details
tokens = tokenizer.tokenize(sample_text)
print(f"First 20 tokens: {tokens[:20]}")

🔧 SETTING UP TOKENIZER...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

✅ Tokenizer loaded: gpt2
Vocabulary size: 50257
Special tokens: ['<|endoftext|>', '<|start|>', '<|end|>', 'Pseudo-Code:', 'Python Code:']

🧪 TOKENIZER TEST:
Sample text length: 102 characters
Tokenized length: 33 tokens
First 20 tokens: ['<|start|>', 'Ċ', 'Pseudo-Code:', 'Ċ', 'in', 'Ġthe', 'Ġfunction', 'Ġg', 'cd', '(', 'a', ',', 'b', '=', 'integ', 'ers', ')', 'ĊĊ', 'Python Code:', 'Ċ']


In [6]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import json

def load_and_prepare_model(model_name="gpt2"):
    print(f"\n🔧 LOADING MODEL: {model_name}")

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )

    # Resize token embeddings for new special tokens
    model.resize_token_embeddings(len(tokenizer))

    # LoRA configuration
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,  # Rank
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["c_attn", "c_proj", "c_fc"]  # GPT-2 modules
    )

    # Apply LoRA
    model = get_peft_model(model, lora_config)

    print("✅ Model loaded and LoRA applied")
    model.print_trainable_parameters()

    return model

def tokenize_function(examples):
    """Tokenize the dataset"""
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    # For causal LM, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

print("📁 LOADING PROCESSED DATASET...")
# Load the processed dataset
with open('pseudo2python_dataset.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

# Tokenize the dataset
print("🔤 TOKENIZING DATASET...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Split dataset (90% train, 10% validation)
train_test_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"✅ Dataset prepared:")
print(f"   Training samples: {len(train_dataset)}")
print(f"   Validation samples: {len(eval_dataset)}")
print(f"   Total samples: {len(train_dataset) + len(eval_dataset)}")

# Load model
model = load_and_prepare_model()

📁 LOADING PROCESSED DATASET...
🔤 TOKENIZING DATASET...


Map:   0%|          | 0/122574 [00:00<?, ? examples/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Dataset prepared:
   Training samples: 110316
   Validation samples: 12258
   Total samples: 122574

🔧 LOADING MODEL: gpt2


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


✅ Model loaded and LoRA applied
trainable params: 1,179,648 || all params: 125,622,528 || trainable%: 0.9390


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [7]:
# COMPREHENSIVE FIX - Addressing all issues at once
print("🎯 COMPREHENSIVE FIX - RESETTING EVERYTHING")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import json

# Step 1: Reload everything from scratch
print("🔄 STEP 1: RELOADING BASE MODEL AND TOKENIZER...")

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Reload base model with proper settings
base_model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    torch_dtype=torch.float16,
)

print("✅ Base model loaded")

# Step 2: Apply LoRA with explicit settings
print("🔄 STEP 2: APPLYING LoRA CONFIGURATION...")

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj", "c_fc"],
)

model = get_peft_model(base_model, lora_config)

# Explicitly enable gradients for all trainable parameters
for param in model.parameters():
    if param.requires_grad:
        param.requires_grad = True

print("✅ LoRA applied")
model.print_trainable_parameters()

# Step 3: Verify gradient setup
print("🔄 STEP 3: VERIFYING GRADIENT SETUP...")

trainable_count = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable_count += 1
        print(f"✓ Trainable: {name}")

print(f"Total trainable parameters: {trainable_count}")

if trainable_count == 0:
    raise ValueError("❌ CRITICAL: No trainable parameters found!")

# Step 4: Load and prepare dataset with simple tokenization
print("🔄 STEP 4: PREPARING DATASET...")

# Load processed data
with open('pseudo2python_dataset.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

def simple_tokenize(example):
    result = tokenizer(
        example['text'],
        truncation=True,
        max_length=512,
        padding=False,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(simple_tokenize, batched=True)
train_test = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

print(f"✅ Dataset prepared: {len(train_test['train'])} train, {len(train_test['test'])} test")

# Step 5: Create a working data collator
print("🔄 STEP 5: CREATING DATA COLLATOR...")

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,
)

# Step 6: Test forward and backward pass manually
print("🔄 STEP 6: TESTING FORWARD/BACKWARD PASS...")

try:
    # Get a test batch
    test_batch = [train_test['train'][i] for i in range(2)]
    collated = data_collator(test_batch)

    # Move to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    collated = {k: v.to(device) for k, v in collated.items()}
    model.to(device)

    # Test forward pass
    outputs = model(**collated)
    loss = outputs.loss

    # Test backward pass
    loss.backward()

    # Check gradients
    grad_exists = False
    for name, param in model.named_parameters():
        if param.requires_grad and param.grad is not None:
            grad_exists = True
            break

    if grad_exists:
        print("✅ Forward/backward test PASSED!")
    else:
        print("❌ Forward/backward test FAILED - no gradients")

except Exception as e:
    print(f"❌ Forward/backward test failed: {e}")

# Step 7: Training setup
print("🔄 STEP 7: SETTING UP TRAINING...")

training_args = TrainingArguments(
    output_dir="./pseudo2python-output",
    num_train_epochs=3,  # Reduced for testing
    per_device_train_batch_size=2,  # Reduced for stability
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    warmup_steps=100,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    dataloader_drop_last=True,
    gradient_checkpointing=False,  # Disabled for stability
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test['train'],
    eval_dataset=train_test['test'],
    data_collator=data_collator,
)

print("🚀 FINAL ATTEMPT - STARTING TRAINING...")

try:
    # Start training
    trainer.train()

    print("🎉 SUCCESS! TRAINING COMPLETED!")
    trainer.save_model()
    print("💾 Model saved successfully!")

except Exception as e:
    print(f"💥 FINAL FAILURE: {e}")
    print("\nThis setup is not working with the current environment configuration.")
    print("The issue appears to be with the PEFT/Trainer integration.")

🎯 COMPREHENSIVE FIX - RESETTING EVERYTHING
🔄 STEP 1: RELOADING BASE MODEL AND TOKENIZER...
✅ Base model loaded
🔄 STEP 2: APPLYING LoRA CONFIGURATION...
✅ LoRA applied
trainable params: 1,179,648 || all params: 125,619,456 || trainable%: 0.9391
🔄 STEP 3: VERIFYING GRADIENT SETUP...
✓ Trainable: base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight
✓ Trainable: base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight
✓ Trainable: base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight
✓ Trainable: base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight
✓ Trainable: base_model.model.transformer.h.0.mlp.c_fc.lora_A.default.weight
✓ Trainable: base_model.model.transformer.h.0.mlp.c_fc.lora_B.default.weight
✓ Trainable: base_model.model.transformer.h.0.mlp.c_proj.lora_A.default.weight
✓ Trainable: base_model.model.transformer.h.0.mlp.c_proj.lora_B.default.weight
✓ Trainable: base_model.model.transformer.h.1.attn.c_attn.lora_A.default.weight
✓ Trai

Map:   0%|          | 0/122574 [00:00<?, ? examples/s]

✅ Dataset prepared: 110316 train, 12258 test
🔄 STEP 5: CREATING DATA COLLATOR...
🔄 STEP 6: TESTING FORWARD/BACKWARD PASS...
❌ Forward/backward test failed: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).
🔄 STEP 7: SETTING UP TRAINING...
🚀 FINAL ATTEMPT - STARTING TRAINING...
💥 FINAL FAILURE: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

This setup is not working with the current environment configuration.
The issue appears to be with the PEFT/Trainer integration.


In [8]:
# COMPLETE TRAINING SESSION - RELOAD EVERYTHING
print("🔄 RELOADING EVERYTHING FOR COMPLETE TRAINING SESSION")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import json
import os

# Step 1: Reload tokenizer and model
print("🔧 STEP 1: LOADING TOKENIZER AND MODEL...")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Add special tokens
special_tokens = {
    'additional_special_tokens': [
        '<|start|>',
        '<|end|>',
        'Pseudo-Code:',
        'Python Code:'
    ]
}
tokenizer.add_special_tokens(special_tokens)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "gpt2",
    torch_dtype=torch.float16,
)
base_model.resize_token_embeddings(len(tokenizer))

# Apply LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj", "c_fc"],
)

model = get_peft_model(base_model, lora_config)
print("✅ Model loaded and LoRA applied")
model.print_trainable_parameters()

# Step 2: Load and prepare dataset
print("🔧 STEP 2: LOADING DATASET...")
with open('pseudo2python_dataset.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)

def simple_tokenize(example):
    result = tokenizer(
        example['text'],
        truncation=True,
        max_length=512,
        padding=False,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(simple_tokenize, batched=True)
train_test = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

print(f"✅ Dataset loaded: {len(train_test['train'])} train, {len(train_test['test'])} test")

# Step 3: Create data collator
print("🔧 STEP 3: CREATING DATA COLLATOR...")

class SimpleDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        # Extract input_ids and labels
        input_ids = [feature['input_ids'] for feature in features]
        labels = [feature['labels'] for feature in features]

        # Find max length in this batch
        max_len = max(len(seq) for seq in input_ids)

        # Pad sequences
        padded_input_ids = []
        padded_labels = []
        attention_masks = []

        for i, seq in enumerate(input_ids):
            padding_length = max_len - len(seq)

            # Pad input_ids with pad_token_id
            padded_seq = seq + [self.tokenizer.pad_token_id] * padding_length
            padded_input_ids.append(padded_seq)

            # Pad labels with -100 (ignore index)
            label_seq = labels[i] + [-100] * padding_length
            padded_labels.append(label_seq)

            # Create attention mask (1 for real tokens, 0 for padding)
            attention_mask = [1] * len(seq) + [0] * padding_length
            attention_masks.append(attention_mask)

        # Convert to tensors
        batch = {
            'input_ids': torch.tensor(padded_input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_masks, dtype=torch.long),
            'labels': torch.tensor(padded_labels, dtype=torch.long)
        }

        return batch

simple_collator = SimpleDataCollator(tokenizer)

# Test the collator
test_features = [train_test['train'][i] for i in range(2)]
collated = simple_collator(test_features)
print(f"✅ Collator test passed! Shapes: {collated['input_ids'].shape}")

# Step 4: Setup training
print("🔧 STEP 4: SETTING UP TRAINING...")

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {device}")

batch_size = 8 if device.type == 'cuda' else 4

train_loader = DataLoader(
    train_test['train'],
    batch_size=batch_size,
    collate_fn=simple_collator,
    shuffle=True,
    num_workers=0
)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
model.train()
model.to(device)

# Step 5: Start training
print("🚀 STEP 5: STARTING TRAINING...")
print(f"Training on {len(train_loader)} batches (batch size: {batch_size})")

for epoch in range(3):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/3")

    for batch_idx, batch in enumerate(progress_bar):
        try:
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            if batch_idx % 50 == 0:
                progress_bar.set_postfix({
                    "loss": f"{loss.item():.4f}",
                    "avg_loss": f"{total_loss/(batch_idx+1):.4f}"
                })

        except Exception as e:
            print(f"❌ Error in batch {batch_idx}: {e}")
            continue

    avg_loss = total_loss / len(train_loader)
    print(f"📊 Epoch {epoch+1} completed - Average Loss: {avg_loss:.4f}")

print("✅ TRAINING COMPLETED!")

# Step 6: Save model
print("💾 SAVING MODEL...")
os.makedirs("./pseudo2python-output", exist_ok=True)
model.save_pretrained("./pseudo2python-output")
tokenizer.save_pretrained("./pseudo2python-output")

print("🎉 SUCCESS! Model trained and saved to ./pseudo2python-output/")

🔄 RELOADING EVERYTHING FOR COMPLETE TRAINING SESSION
🔧 STEP 1: LOADING TOKENIZER AND MODEL...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


✅ Model loaded and LoRA applied
trainable params: 1,179,648 || all params: 125,622,528 || trainable%: 0.9390
🔧 STEP 2: LOADING DATASET...


Map:   0%|          | 0/122574 [00:00<?, ? examples/s]

✅ Dataset loaded: 110316 train, 12258 test
🔧 STEP 3: CREATING DATA COLLATOR...
✅ Collator test passed! Shapes: torch.Size([2, 32])
🔧 STEP 4: SETTING UP TRAINING...
🖥️ Using device: cuda
🚀 STEP 5: STARTING TRAINING...
Training on 13790 batches (batch size: 8)


...

📊 Epoch 2 completed - Average Loss: 2.4481


Epoch 3/3:   0%|          | 0/13790 [00:00<?, ?it/s]

📊 Epoch 3 completed - Average Loss: 2.3851
✅ TRAINING COMPLETED!
💾 SAVING MODEL...


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


🎉 SUCCESS! Model trained and saved to ./pseudo2python-output/


In [9]:
# Install required dependencies
print("📦 INSTALLING REQUIRED DEPENDENCIES...")
!pip install evaluate codebleu

📦 INSTALLING REQUIRED DEPENDENCIES...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.2/546.2 kB 17.0 MB/s eta 0:00:00


In [10]:
# Step 6 — Simplified Evaluation
print("📊 STEP 6: EVALUATING THE TRAINED MODEL...")

import math
import numpy as np
from tqdm.auto import tqdm

def simplified_evaluate_model(model, tokenizer, eval_dataset):
    print("🧪 Running simplified evaluation...")

    model.eval()
    predictions = []
    references = []

    # Use a smaller subset for faster evaluation
    eval_subset = eval_dataset.select(range(min(500, len(eval_dataset))))

    print("🔍 Generating predictions...")
    for example in tqdm(eval_subset, desc="Evaluating"):
        try:
            # Extract pseudo-code part for generation
            text = example['text']
            pseudo_start = text.find("Pseudo-Code:") + len("Pseudo-Code:")
            code_start = text.find("Python Code:")

            if pseudo_start != -1 and code_start != -1:
                prompt = text[:code_start + len("Python Code:")]

                # Generate code
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                with torch.no_grad():
                    outputs = model.generate(
                        inputs.input_ids,
                        max_length=512,
                        num_return_sequences=1,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=tokenizer.eos_token_id,
                        eos_token_id=tokenizer.eos_token_id
                    )

                generated = tokenizer.decode(outputs[0], skip_special_tokens=False)

                # Extract generated code
                if "Python Code:" in generated:
                    generated_code = generated.split("Python Code:")[1].split("<|end|>")[0].strip()
                else:
                    generated_code = ""

                # Extract reference code
                reference_code = text.split("Python Code:")[1].split("<|end|>")[0].strip()

                predictions.append(generated_code)
                references.append(reference_code)

        except Exception as e:
            continue

    print(f"✅ Generated {len(predictions)} predictions")

    # Calculate basic metrics
    results = {}

    # Calculate exact match accuracy
    exact_matches = sum(1 for pred, ref in zip(predictions, references) if pred.strip() == ref.strip())
    results['exact_match'] = exact_matches / len(predictions) if predictions else 0
    print(f"✅ Exact Match Accuracy: {results['exact_match']:.4f}")

    # Calculate average length
    avg_pred_length = np.mean([len(pred) for pred in predictions]) if predictions else 0
    avg_ref_length = np.mean([len(ref) for ref in references]) if references else 0
    results['avg_prediction_length'] = avg_pred_length
    results['avg_reference_length'] = avg_ref_length
    print(f"✅ Avg Prediction Length: {avg_pred_length:.1f} chars")
    print(f"✅ Avg Reference Length: {avg_ref_length:.1f} chars")

    # Calculate perplexity
    try:
        eval_loss = calculate_perplexity(model, tokenizer, eval_subset)
        results['perplexity'] = math.exp(eval_loss)
        results['eval_loss'] = eval_loss
        print(f"✅ Perplexity: {results['perplexity']:.2f}")
        print(f"✅ Evaluation Loss: {eval_loss:.4f}")
    except Exception as e:
        print(f"❌ Perplexity calculation failed: {e}")
        results['perplexity'] = float('inf')
        results['eval_loss'] = float('inf')

    # Sample outputs for human evaluation
    results['sample_predictions'] = []
    print("\n🔍 SAMPLE PREDICTIONS:")
    for i in range(min(5, len(predictions))):
        sample = {
            'input': eval_subset[i]['text'].split("Pseudo-Code:")[1].split("Python Code:")[0].strip()[:100],
            'prediction': predictions[i][:200],
            'reference': references[i][:200]
        }
        results['sample_predictions'].append(sample)

        print(f"\nSample {i+1}:")
        print(f"Input: {sample['input']}...")
        print(f"Generated: {sample['prediction']}...")
        print(f"Reference: {sample['reference']}...")
        print("-" * 50)

    return results, predictions, references

def calculate_perplexity(model, tokenizer, eval_dataset):
    """Calculate perplexity on evaluation dataset"""
    model.eval()
    total_loss = 0
    total_tokens = 0

    for example in tqdm(eval_dataset, desc="Calculating perplexity"):
        try:
            inputs = tokenizer(example['text'], return_tensors="pt", truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            inputs["labels"] = inputs["input_ids"].clone()

            with torch.no_grad():
                outputs = model(**inputs)
                loss = outputs.loss

            total_loss += loss.item() * inputs["input_ids"].size(1)
            total_tokens += inputs["input_ids"].size(1)

        except Exception as e:
            continue

    return total_loss / total_tokens if total_tokens > 0 else float('inf')

# Run evaluation
print("🚀 Starting evaluation...")
eval_results, predictions, references = simplified_evaluate_model(model, tokenizer, train_test['test'])

# Save evaluation results
with open('evaluation_results.json', 'w') as f:
    json.dump(eval_results, f, indent=2)

print("💾 Evaluation results saved to evaluation_results.json")

📊 STEP 6: EVALUATING THE TRAINED MODEL...
🚀 Starting evaluation...
🧪 Running simplified evaluation...
🔍 Generating predictions...


Evaluating:   0%|          | 0/500 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✅ Generated 500 predictions
✅ Exact Match Accuracy: 0.0000
✅ Avg Prediction Length: 1512.0 chars
✅ Avg Reference Length: 23.6 chars


Calculating perplexity:   0%|          | 0/500 [00:00<?, ?it/s]

✅ Perplexity: 9.17
✅ Evaluation Loss: 2.2156

🔍 SAMPLE PREDICTIONS:

Sample 1:
Input: count = integer with value 1...
Generated: int count = 1;
 strugg = int(1), std::set<int>, std::set<int>();
, encount;
.begin() { encount++; }
!, encount;
®, encount;
 misunder;
 conduc;
 oun;
 misunder;
 challeng;
 encount;
;
 encount;
 misun...
Reference: int count = 1;...
--------------------------------------------------

Sample 2:
Input: let int on = n...
Generated: int on = n;
, x = n;
The following integer on = n;
 to misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder misunder m...
Reference: int on = n;...
--------------------------------------------------

Sample 3:
Input: declare string s1 = ""...
Generated: string s1 = "";
:string();
 an = string(s1);
 (void)an;
 and misunderccording( misunderccording( misunderccording( misunderccording( misunderccording( misunderccording( misunderccording( misunderccord...
